# 03. AWQ 양자화 — EXAONE-4.0-1.2B

**전략:** AWQ(Activation-aware Weight Quantization)는 GPTQ와 같은 4-bit이지만  
활성화 분포를 고려해서 중요한 가중치를 보호하므로 **정확도 손실이 더 적음**.  
vLLM 0.14.1에서 AWQ 커널을 최적화해두어서 추론 속도도 빠른 편.

| 방식 | 정확도 유지 | 추론 속도 | 모델 크기 |
|------|-----------|----------|----------|
| GPTQ W4A16 | 보통 | 보통 | 작음 |
| **AWQ W4A16** | **좋음** | **좋음** | 작음 |
| FP8 W8A8 | 매우 좋음 | 최고(L4) | 보통 |

**환경:** Google Colab Free (T4 16GB)

## 1. 패키지 설치

> ⚠️ 설치 후 **런타임 재시작** 필요

In [ ]:
# autoawq: AWQ 양자화 공식 라이브러리
!pip install -q \
    autoawq \
    "transformers>=4.54.0" \
    datasets \
    accelerate \
    sentencepiece

print("\n✅ 설치 완료! 런타임을 재시작하세요")

## 2. 설정값

In [ ]:
# ============================================================
# AWQ 실험 설정
# ============================================================

MODEL_ID = "LGAI-EXAONE/EXAONE-4.0-1.2B"
OUT_DIR  = "./model_awq"

# AWQ 양자화 설정
AWQ_CONFIG = {
    "zero_point": True,          # Zero-point 양자화 사용 (정확도 향상)
    "q_group_size": 128,         # 그룹 크기 (128이 표준)
    "w_bit": 4,                  # 4-bit 가중치
    "version": "GEMM",           # vLLM 호환 커널 버전
}

DATASET_ID    = "LGAI-EXAONE/MANTA-1M"
DATASET_SPLIT = "train"
NUM_CALIBRATION_SAMPLES = 512
MAX_SEQUENCE_LENGTH     = 1024

ZIP_NAME = "submit_awq"

print("=" * 50)
print(f"모델        : {MODEL_ID}")
print(f"AWQ 설정    : {AWQ_CONFIG}")
print(f"캘리브 샘플 : {NUM_CALIBRATION_SAMPLES}")
print(f"시퀀스 길이 : {MAX_SEQUENCE_LENGTH}")
print("=" * 50)

## 3. 모델 로드

In [ ]:
from awq import AutoAWQForCausalLM
from transformers import AutoTokenizer

print("토크나이저 로드 중...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, trust_remote_code=True)

print("모델 로드 중... (AWQ 전용 로더 사용)")
model = AutoAWQForCausalLM.from_pretrained(
    MODEL_ID,
    trust_remote_code=True,
    safetensors=True,
)
print("✅ 로드 완료")

## 4. 캘리브레이션 데이터 준비

In [ ]:
from datasets import load_dataset

print(f"데이터 로드: {NUM_CALIBRATION_SAMPLES}샘플")
ds = load_dataset(DATASET_ID, split=f"{DATASET_SPLIT}[:{NUM_CALIBRATION_SAMPLES}]")

# AWQ는 텍스트 리스트를 바로 받음
def preprocess(example):
    return {
        "text": tokenizer.apply_chat_template(
            example["conversations"],
            add_generation_prompt=True,
            tokenize=False,
        )
    }

ds = ds.map(preprocess)
calib_data = ds["text"]  # AWQ는 텍스트 리스트로 전달
print(f"✅ 데이터 준비: {len(calib_data)}개")

## 5. AWQ 양자화 실행

> T4에서 약 10~20분 소요

In [ ]:
import time

print(f"AWQ 양자화 시작...")
start_time = time.time()

model.quantize(
    tokenizer,
    quant_config=AWQ_CONFIG,
    calib_data=calib_data,
    max_calib_seq_len=MAX_SEQUENCE_LENGTH,
)

elapsed = time.time() - start_time
print(f"\n✅ AWQ 완료! 소요 시간: {elapsed/60:.1f}분")

## 6. 저장 & 제출 파일 생성

In [ ]:
import os, shutil
from pathlib import Path

os.makedirs(OUT_DIR, exist_ok=True)
print(f"모델 저장 중: {OUT_DIR}")

# AWQ는 save_quantized 사용
model.save_quantized(OUT_DIR, safetensors=True)
tokenizer.save_pretrained(OUT_DIR)

total_size = sum(f.stat().st_size for f in Path(OUT_DIR).iterdir()) / 1024**3
print(f"\n총 크기: {total_size:.2f} GB")

# config.json 확인
import json
with open(f"{OUT_DIR}/config.json") as f:
    cfg = json.load(f)
print(f"quantization_config: {cfg.get('quantization_config', '없음')}")
print("\n✅ 저장 완료")

In [ ]:
shutil.make_archive(base_name=ZIP_NAME, format="zip", root_dir=".", base_dir=OUT_DIR)
zip_size = Path(f"{ZIP_NAME}.zip").stat().st_size / 1024**3
print(f"✅ {ZIP_NAME}.zip 생성 완료 ({zip_size:.2f} GB)")

## 7. Sanity Check

In [ ]:
from awq import AutoAWQForCausalLM
from transformers import AutoTokenizer
import torch

print("저장된 AWQ 모델 추론 테스트...")
model_check = AutoAWQForCausalLM.from_quantized(
    OUT_DIR,
    trust_remote_code=True,
    safetensors=True,
)
tok_check = AutoTokenizer.from_pretrained(OUT_DIR, trust_remote_code=True)

messages = [{"role": "user", "content": "대한민국의 수도는"}]
inputs = tok_check.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt")
inputs = inputs.to(model_check.device)

with torch.no_grad():
    outputs = model_check.generate(inputs, max_new_tokens=50, do_sample=False)

response = tok_check.decode(outputs[0][inputs.shape[-1]:], skip_special_tokens=True)
print(f"모델 응답: {response}")
print("\n✅ AWQ 모델 정상 작동!")

---
## 📊 실험 결과 기록

| 실험 | 방식 | 샘플 수 | group_size | LB Score | 비고 |
|------|------|---------|------------|----------|------|
| AWQ 기본 | W4A16 GEMM | 512 | 128 | - | |

---
## ⏭️ 비교 결론

3개 실험(GPTQ / FP8 / AWQ) LB 점수 비교 후:
- 가장 높은 방식을 메인으로 채택
- 채택된 방식 위에 `04_qlora_sft.ipynb`로 QLoRA SFT 적용